## Lakehouse 4 - Data Wrangler

This notebook is continues the learning of building a lakehouse by getting hands-on with data wrangler. The fact table built previously summarizes the min/max/close prices on a daily basis. Using data wrangler, we can easily create another summarized view that summarizes the data to the minute, which is written to the *stocks_minute_agg* table. Additionally, data wrangler can be used to do preprocessing of the values to reject invalid data. This 'silver' table (in medallion architecture) would typically be used to then feed into other models, including the data science module. 

This framework can also be used to summarize to the hour: note that table schema (the *stocks_hour_agg* table) is created to support this view. The advantage of these aggregation tables are that they support business cases where different views of the data might be needed.

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *
import datetime
import time
from datetime import datetime
from datetime import timedelta

sourceTableName = 'raw_stock_data'

In [ ]:
def create_minute_aggregation_table():
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS stocks_minute_agg (
            Symbol VARCHAR(5) NOT NULL
            ,Datestamp DATE 
            ,Hour INT
            ,Minute INT
            ,MinPrice DOUBLE 
            ,MaxPrice DOUBLE 
            ,LastPrice DOUBLE
            )
        USING DELTA
        """)

create_minute_aggregation_table()

In [ ]:
def create_hour_aggregation_table():
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS stocks_hour_agg (
            Symbol VARCHAR(5) NOT NULL
            ,Datestamp DATE 
            ,Hour INT
            ,MinPrice DOUBLE 
            ,MaxPrice DOUBLE 
            ,LastPrice DOUBLE
            )
        USING DELTA
        """)

create_hour_aggregation_table()

In [ ]:
from pyspark.sql import Row

def createAnomalyDataframe():

    df = spark.createDataFrame([
        Row(symbol="WHO", price=250.55, timestamp='2023-10-01 04:40:00')
        ,Row(symbol="WHAT", price=432.23, timestamp='2023-10-01 04:40:00')
        ,Row(symbol=None, price=124.33, timestamp='2023-10-06 00:00:00')
        ,Row(symbol="WHY", price=103.11, timestamp='2023-10-01 04:40:00')
        ,Row(symbol="BCUZ", price=12.0, timestamp='2023-10-02 00:00:00')
        ,Row(symbol="IDK", price=0.0, timestamp='2023-10-04 17:00:00')
        ,Row(symbol="IDGD", price=0.0, timestamp='2023-10-04 17:00:00')
        ,Row(symbol="TMRW", price=None, timestamp='2023-10-06 00:00:00')
        ,Row(symbol="TDY", price=None, timestamp='2023-10-06 00:00:00')
    ])

    return df

anomaly_df = createAnomalyDataframe();

In [ ]:
# to insert the new data, we'll merge the dataframe with the fact table
# for existing records, update the high/low/close price of the stock
# for new records, insert a new row with the current high/low/close

from delta.tables import *

def merge_minute_agg(df):
    stocks_minute_agg_table = DeltaTable.forName(spark, "stocks_minute_agg")

    stocks_minute_agg_table.alias('table') \
    .merge(
        df.alias('updates'),
        'table.symbol = updates.symbol and table.datestamp = updates.datestamp and table.hour = updates.hour and table.minute = updates.minute'
    ) \
    .whenMatchedUpdate(set =
        {
            "MinPrice": "CASE WHEN table.MinPrice < updates.price_min THEN table.MinPrice ELSE updates.price_min END"
            ,"MaxPrice": "CASE WHEN table.MaxPrice > updates.price_max THEN table.MaxPrice ELSE updates.price_max END"
            ,"LastPrice": "updates.price_last"
        }
    ) \
    .whenNotMatchedInsert(values =
        {
            "Symbol": "updates.symbol"
            ,"Datestamp": "updates.datestamp"
            ,"Hour": "updates.hour"
            ,"Minute": "updates.minute"
            ,"MinPrice": "updates.price_min"
            ,"MaxPrice": "updates.price_max"
            ,"LastPrice": "updates.price_last"
        }
    ) \
    .execute()


In [ ]:
# to insert the new data, we'll merge the dataframe with the fact table
# for existing records, update the high/low/close price of the stock
# for new records, insert a new row with the current high/low/close

from delta.tables import *

def merge_hour_agg(df):
    stocks_minute_agg_table = DeltaTable.forName(spark, "stocks_hour_agg")

    stocks_minute_agg_table.alias('table') \
    .merge(
        df.alias('updates'),
        'table.symbol = updates.symbol and table.datestamp = updates.datestamp and table.hour = updates.hour'
    ) \
    .whenMatchedUpdate(set =
        {
            "MinPrice": "CASE WHEN table.MinPrice < updates.price_min THEN table.MinPrice ELSE updates.price_min END"
            ,"MaxPrice": "CASE WHEN table.MaxPrice > updates.price_max THEN table.MaxPrice ELSE updates.price_max END"
            ,"LastPrice": "updates.price_last"
        }
    ) \
    .whenNotMatchedInsert(values =
        {
            "Symbol": "updates.symbol"
            ,"Datestamp": "updates.datestamp"
            ,"Hour": "updates.hour"
            ,"MinPrice": "updates.price_min"
            ,"MaxPrice": "updates.price_max"
            ,"LastPrice": "updates.price_last"
        }
    ) \
    .execute()


In [ ]:
# find latest date 

df_watermark = spark.sql(f"SELECT datestamp, hour, minute \
    FROM stocks_minute_agg \
    ORDER BY Datestamp DESC, Hour DESC, Minute DESC LIMIT 1")

if not df_watermark.rdd.isEmpty():
    df_watermark.show()
    maxDate = df_watermark.first()["datestamp"]
    maxHour = df_watermark.first()["hour"]
    maxMinute = df_watermark.first()["minute"]
    cutoff_datetime = datetime(maxDate.year, maxDate.month, maxDate.day, maxHour, maxMinute, 0)
else:
    print('Table is empty, using default date.')
    cutoff_datetime = '2000-01-01 00:00:00'

# manually specify a cutoff date
#cutoff_datetime = '2023-11-27 23:59:50'

print(f"Cutoff date: {cutoff_datetime}")


In [ ]:
# get new stock data to ingest, starting at watermark
# limit is arbitrary; limited primarily for demo purposes

df_stocks = spark.sql(f"SELECT symbol, price, timestamp FROM {sourceTableName} \
    WHERE timestamp >= '{cutoff_datetime}' \
    ORDER BY timestamp ASC LIMIT 5000000")
df_stocks.show()


#### Add data wrangler code here to:
1. Drop missing values from symbol and price
2. Drop values from price where price = 0

See commented-out sample for reference of output.

In [ ]:
# add data wrangler here


In [ ]:
# # Code generated by Data Wrangler for PySpark DataFrame

# def remove_invalid_rows(df):
#     # Drop rows with missing data in columns: 'symbol', 'price'
#     df = df.dropna(subset=['symbol', 'price'])
#     # Filter rows based on column: 'price'
#     df = df.filter(~(df['price'] == 0))
#     return df

# df_stocks_clean = remove_invalid_rows(df_stocks)
# display(df_stocks_clean)

#### Symbol/Date/Hour/Minute Aggregation Section

Add data wrangler code here to:
1. Convert column timestamp to a timestamp type
2. Add new column datestamp, based on timestamp (date without a time component)
3. Add new columm hour, based on timestamp hour value
4. Add new column minute, based on timestamp minute value
5. Group By symbol, datestamp, hour, and minute
    1. Add aggregations for minimum price, maximum price, and last price

See commented-out sample for reference of output.

In [ ]:
# add data wrangler here


In [ ]:
# # Code generated by Data Wrangler for PySpark DataFrame

# from datetime import datetime
# from pyspark.sql import functions as F
# from pyspark.sql import types as T

# def aggregate_data(df_stocks_clean):
#     # Change column type to datetime64[ns] for column: 'timestamp'
#     df_stocks_clean = df_stocks_clean.withColumn('timestamp', df_stocks_clean['timestamp'].cast(T.TimestampType()))
#     # Derive column 'datestamp' from column: 'timestamp'
    
#     # Transform based on the following examples:
#     #    timestamp                  Output
#     # 1: 2023-12-01T13:22:00.938 => "2023-12-01"
#     udf_fn = F.udf(lambda v : v.strftime("%Y-%m-%d"), T.StringType())
#     df_stocks_clean = df_stocks_clean.withColumn("datestamp", udf_fn(F.col("timestamp")))
#     # Derive column 'hour' from column: 'timestamp'
    
#     def hour(timestamp):
#         """
#         Transform based on the following examples:
#            timestamp                  Output
#         1: 2023-12-01T13:22:00.938 => "13"
#         """
#         number1 = timestamp.hour
#         return f"{number1:01.0f}"
    
#     udf_fn = F.udf(lambda v : hour(v), T.StringType())
#     df_stocks_clean = df_stocks_clean.withColumn("hour", udf_fn(F.col("timestamp")))
#     # Derive column 'minute' from column: 'timestamp'
    
#     def minute(timestamp):
#         """
#         Transform based on the following examples:
#            timestamp                  Output
#         1: 2023-12-01T13:22:00.938 => "22"
#         """
#         number1 = timestamp.minute
#         return f"{number1:01.0f}"
    
#     udf_fn = F.udf(lambda v : minute(v), T.StringType())
#     df_stocks_clean = df_stocks_clean.withColumn("minute", udf_fn(F.col("timestamp")))
#     # Performed 3 aggregations grouped on columns: 'symbol', 'datestamp' and 2 other columns
#     df_stocks_clean = df_stocks_clean.groupBy('symbol', 'datestamp', 'hour', 'minute').agg(F.max('price').alias('price_max'), F.min('price').alias('price_min'), F.last('price').alias('price_last'))
#     df_stocks_clean = df_stocks_clean.dropna()
#     df_stocks_clean = df_stocks_clean.sort(df_stocks_clean['symbol'].asc(), df_stocks_clean['datestamp'].asc(), df_stocks_clean['hour'].asc(), df_stocks_clean['minute'].asc())
#     return df_stocks_clean

# df_stocks_agg_minute = aggregate_data(df_stocks_clean)
# display(df_stocks_agg_minute)

In [ ]:
# write the data to the stocks_minute_agg table

merge_minute_agg(df_stocks_agg_minute)

#### Symbol/Date/Hour Aggregation Section

Add data wrangler code here to:
1. Group By symbol, datestamp, hour
    1. Add aggregations for minimum price, maximum price, and last price

In the generated code, modify the alias' to keep price_max, price_min, price_last to the same name. Because we're aggregating the data a second time, the default naming will try to set the names to price_max_max, etc.

See commented-out sample for reference of output.

In [ ]:
# add data wrangler here


In [ ]:
# # Code generated by Data Wrangler for PySpark DataFrame

# from pyspark.sql import functions as F

# def aggregate_data_minute(df_stocks_agg_minute):
#     # Performed 3 aggregations grouped on columns: 'symbol', 'datestamp', 'hour'
#     df_stocks_agg_minute = df_stocks_agg_minute.groupBy('symbol', 'datestamp', 'hour').agg(
#         F.max('price_max').alias('price_max'), 
#         F.min('price_min').alias('price_min'), 
#         F.last('price_last').alias('price_last'))
#     df_stocks_agg_minute = df_stocks_agg_minute.dropna()
#     df_stocks_agg_minute = df_stocks_agg_minute.sort(df_stocks_agg_minute['symbol'].asc(), df_stocks_agg_minute['datestamp'].asc(), df_stocks_agg_minute['hour'].asc())
#     return df_stocks_agg_minute

# df_stocks_agg_hour = aggregate_data_minute(df_stocks_agg_minute)
# display(df_stocks_agg_hour)

In [ ]:
# uncomment the line below to write the data to the stocks_hour_agg table

# merge_hour_agg(df_stocks_agg_hour)

#### Handy SQL Commands for testing
Use this section for testing, cleaning out tables to re-run, etc.

In [ ]:
df = spark.sql("SELECT min(datestamp) as mindate, max(datestamp) as maxdate FROM StocksLakehouse.stocks_minute_agg")
df.show()

df = spark.sql("SELECT * FROM StocksLakehouse.stocks_minute_agg ORDER BY datestamp DESC, hour DESC, minute DESC LIMIT 1000")
df.show()

In [ ]:
df = spark.sql("SELECT min(datestamp) as mindate, max(datestamp) as maxdate FROM StocksLakehouse.stocks_hour_agg")
df.show()

df = spark.sql("SELECT * FROM StocksLakehouse.stocks_hour_agg ORDER BY datestamp DESC, hour DESC LIMIT 1000")
df.show()

In [ ]:
def deleteRowsFromTable():
    spark.sql("DELETE FROM StocksLakehouse.stocks_minute_agg")
    spark.sql("DELETE FROM StocksLakehouse.stocks_hour_agg")

deleteRowsFromTable()